# Internal API
This notebook is meant to provide insight into the inner workings of TrajectoryOptimization.jl. This is geared towards people who are interested in using TrajectoryOptimization to create a solver. It provides examples of how to use the methods provided in TrajectoryOptimization to evaluate the dynamics, objective, and constraints. It assumes a decent level of profficiency with Julia, a good knowledge of trajectory optimization, and some familiarity with the basics of TrajectoryOptimization.

In [1]:
import Pkg; Pkg.activate(@__DIR__)
using TrajectoryOptimization
using RobotDynamics
using StaticArrays, LinearAlgebra
using Rotations
using RobotZoo: Quadrotor
using Test
using BenchmarkTools
const TO = TrajectoryOptimization
const RD = RobotDynamics

 Activating environment at `~/.julia/dev/TrajectoryOptimization/examples/Project.toml`
┌ Info: Precompiling TrajectoryOptimization [c79d492b-0548-5874-b488-5a62c1d9d0ca]
└ @ Base loading.jl:1278


RobotDynamics

# Create a Problem

In [2]:
model = Quadrotor()
n,m = size(model)           # number of states and controls
n̄ = state_diff_size(model)  # size of error state
N = 51                      # number of knot points
tf = 5.0                    # final time

# initial and final conditions
x0 = RBState([1,2,1], UnitQuaternion(I), zeros(3), zeros(3))
xf = RBState([0,0,2], UnitQuaternion(I), zeros(3), zeros(3))

# objective
Q = Diagonal(@SVector fill(0.1, n))
R = Diagonal(@SVector fill(0.01, m))
Qf = Diagonal(@SVector fill(100.0, n))
obj = LQRObjective(Q,R,Qf,xf,N)

# constraints
cons = ConstraintList(n,m,N)
add_constraint!(cons, BoundConstraint(n,m, u_min=zeros(4), u_max=fill(10.0,4)), 1:N-1)
add_constraint!(cons, CircleConstraint(n, SA_F64[1,2], SA_F64[1,2], SA[0.1,0.1]), 1:N-1)
add_constraint!(cons, GoalConstraint(xf, SA[1,2,3]), N)

# problem
prob = Problem(model, obj, xf, tf, x0=x0, constraints=cons);

In [3]:
# initialize the controls
u0 = zeros(model)[2]                        # get hover control
initial_controls!(prob, u0)                 # set all time-steps to the same
initial_controls!(prob, [u0 for k = 1:N-1]) # use a vector of initial controls
initial_controls!(prob, fill(u0[1],m,N-1))  # use a matrix of initial controls

# Simulating the dynamics
To simulate the dynamics forward from the initial state, use the `rollout!` method:

In [4]:
# simulate the system forward
rollout!(prob)
@test state(prob.Z[1]) == prob.x0
@test states(prob)[end] ≈ prob.x0

# alternative method
rollout!(TO.integration(prob), prob.model, prob.Z, prob.x0)
@test state(prob.Z[1]) == prob.x0
@test states(prob)[end] ≈ prob.x0

# change control so that the state changes
u0 += [1,0,1,0]*1e-2
initial_controls!(prob, u0)
rollout!(TO.integration(prob), prob.model, prob.Z, prob.x0)
states(prob)[end]

13-element SArray{Tuple{13},Float64,1,13} with indices SOneTo(13):
 1.0
 2.0
 1.5001876006215789
 0.7209485318742003
 0.0
 0.0
 0.6929869546740692
 0.0
 0.0
 0.20003656032918066
 0.0
 0.0
 0.6124999999999969

## Computing the dynamics Jacobians
For normal models, all you just need to pre-allocate some `DynamicsExpansion` types

In [5]:
D = [TO.DynamicsExpansion{Float64}(n,n̄,m) for k = 1:N-1]
TO.dynamics_expansion!(TO.integration(prob), D, model, prob.Z)

 For a `LieGroupModel`, we need to first compute the error-state Jacobians, and then use those along with the Jacobians we computed before to calculate the Jacobians on the error dynamics. Note that we allocate one extra for use as a temporary array for intermediate matrix multiplications.

In [6]:
G = [SizedMatrix{n,n̄}(zeros(n,n̄)) for k = 1:N+1]
RD.state_diff_jacobian!(G, model, prob.Z)
TO.error_expansion!(D, model, G)

We can extract the dynamics Jacobians as each time step using `TO.error_expansion`. Note that the size of these Jacobians match the size of the error state (12).

In [7]:
A,B = TO.error_expansion(D[1], model)
display(A)
display(B)

12×12 SizedArray{Tuple{12,12},Float64,2,2,Array{Float64,2}} with indices SOneTo(12)×SOneTo(12):
 1.0  0.0  0.0   0.0        0.0985     0.0  0.1  …   0.00164167   0.0
 0.0  1.0  0.0  -0.0985     0.0        0.0  0.0      0.0          0.0
 0.0  0.0  1.0   0.0        0.0        0.0  0.0      0.0          0.0
 0.0  0.0  0.0   1.0        0.0006125  0.0  0.0      1.28714e-5   0.0
 0.0  0.0  0.0  -0.0006125  1.0        0.0  0.0      0.05         0.0
 0.0  0.0  0.0   0.0        0.0        1.0  0.0  …   0.0          0.05
 0.0  0.0  0.0   0.0        1.97       0.0  1.0      0.04925      0.0
 0.0  0.0  0.0  -1.97       0.0        0.0  0.0      5.0276e-6    0.0
 0.0  0.0  0.0   0.0        0.0        0.0  0.0      0.0          5.0276e-6
 0.0  0.0  0.0   0.0        0.0        0.0  0.0     -0.000452717  0.0
 0.0  0.0  0.0   0.0        0.0        0.0  0.0  …   1.0          0.0
 0.0  0.0  0.0   0.0        0.0        0.0  0.0      0.0          1.0

12×4 SizedArray{Tuple{12,4},Float64,2,2,Array{Float64,2}} with indices SOneTo(12)×SOneTo(4):
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.01         0.01         0.01         0.01
 -2.95492e-5   0.190217     2.95492e-5  -0.190217
 -0.190217    -2.95492e-5   0.190217     2.95492e-5
  0.0153125   -0.0153125    0.0153125   -0.0153125
 -0.124909     7.6507e-5    0.124909    -7.6507e-5
 -7.6507e-5   -0.124909     7.6507e-5    0.124909
  0.200006     0.199994     0.200006     0.199994
  0.00229639   7.6087      -0.00229639  -7.6087
 -7.6087       0.00229639   7.6087      -0.00229639
  0.6125      -0.6125       0.6125      -0.6125

# Computing the Cost
Getting the cost from the `Problem` type is very easy:

In [8]:
cost(prob)

312.79752892757296

However, under the hood there's a bit more going on. This method actually calls:

In [9]:
TO.cost!(obj, prob.Z);

which computes the cost for each time step, storing it in the vector `J` stored inside the objective. It can be helpful to have access to the vector when debugging the cost, so we provide a getter method:

In [10]:
TO.get_J(obj)'

1×51 Adjoint{Float64,Array{Float64,1}}:
 0.033032  0.0330308  0.0330273  0.0330215  …  0.0337857  0.0339594  311.154

The cost at a single knot point can be computed using `TO.stage_cost`:

In [11]:
TO.stage_cost(obj[1], prob.Z[1])

0.033032003124999994

## Cost Expansion
To store the 2nd order expansion of the cost function we use the `TO.Expansion` type, and the convenient `TO.CostExpansion` constructor:

In [12]:
E0 = TO.CostExpansion(n, m, N)
typeof(E0)

Array{TrajectoryOptimization.Expansion{13,4,Float64},1}

To evaluate the 2nd Order Taylor Series expansion of the cost we can use the following methods:

In [13]:
TO.cost_expansion!(E0, obj, prob.Z)

# this just calls these two functions
TO.cost_gradient!(E0, obj, prob.Z)
TO.cost_hessian!(E0, obj, prob.Z)

# which call these functions each time step
TO.gradient!(E0[1], obj[1], prob.Z[1])
TO.hessian!(E0[1], obj[1], prob.Z[1])

true

The boolean flag returned from `TO.hessian!` specifies that the hessian is constant. Some computation can be saved by leveraging this information. Internally, this is stored in the objective:

In [14]:
obj.const_hess'

1×51 Adjoint{Bool,BitArray{1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

We can force it to recompute using the optional `init` flag:

In [15]:
@btime TO.cost_expansion!($E0, $obj, $(prob.Z), init=true)
@btime TO.cost_expansion!($E0, $obj, $(prob.Z), init=false)

  38.098 μs (0 allocations: 0 bytes)
  2.591 μs (0 allocations: 0 bytes)


The `TO.cost_expansion!` takes one additional argument: `rezero`. If true, this will multiply each of the terms in the Hessian by zero prior to computing the new expansion. By default, the expansions may choose to only modify the elements they know to be non-zero (e.g. Diagonal costs only modify the diagonal). If we end up adding addition terms to the expansion (i.e. augmented Lagrangian penalty terms), our expansion will be incorrect if we don't re-zero:

In [16]:
E0[1].Q[1,2] = 10
TO.cost_expansion!(E0, obj, prob.Z)
@show E0[1].Q[1,2]  # this is incorrect
TO.cost_expansion!(E0, obj, prob.Z, init=true, rezero=true)
@show E0[1].Q[1,2]; # now correct

(E0[1]).Q[1, 2] = 10.0
(E0[1]).Q[1, 2] = 0.0


### Expansion caches (advanced)
TrajectoryOptimization provides methods to differentiate custom nonlinear cost functions automatically with either [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl) or [FiniteDiff.jl](https://github.com/JuliaDiff/FiniteDiff.jl). For finite differencing, we can save compuation and allocations by passing a cache. This can be automatically allocated using `TO.ExpansionCache(cost::CostFunction)`. While the automatic methods (e.g. `_gradient!` and `_hessian!`) shouldn't be modified, the `TO.ExpansionCache` can be overwritten for a custom cost function type where you want to pre-allocated some memory to use while computing the gradient and Hessian of the cost function. 

This function returns a vector of 4 caches: `[grad, hess, grad_term, hess_term]` which are meant to be used by gradient and Hessian computations for the stage and terminal cost functions, respectively. 

### Error State Expansions
If the model is a `LieGroupModel`, like our Quadrotor model, we need to again convert the expansions to be valid for the lower-dimensional error state. This ends up being very similar to what we did for the dynamics. First, we can use the following constructor to initialize a new `QuadraticObjective` to store the error state expansion. In the second line, this constructor will alias `E === E0` if model is not a `LieGroupModel`, saving computation and avoiding unnecessary storage.

In [17]:
E = TO.CostExpansion(n̄, m, N)     # error state Jacobians
E0 = TO.CostExpansion(E, model);  # "normal" Jacobians

We can now compute the expansion on the error state using methods very similar to what we did previously for the dynamics Jacobian. Note that the 3x3 diagonal block for the rotation state is no longer diagonal.

In [18]:
RD.state_diff_jacobian!(G, model, prob.Z)
TO.cost_expansion!(E0, obj, prob.Z, init=true, rezero=true)
TO.error_expansion!(E, E0, model, prob.Z, G);
E[N-1].Q[4:6,4:6]

3×3 Array{Float64,2}:
 0.00741626   2.65474e-19  0.0
 2.84866e-19  0.00741626   0.0
 0.0          0.0          0.00741626

# Constraints
Constraints in TrajectoryOptimization can get a little complicated. Let's start by making a couple important points:
* Each constraint is defined by an `AbstractConstraint`. Specific per-timestep constraints should inherit from it's subtypes: `StageConstraint`, `ControlConstraint`, or `StageConstraint`.
* Each constraint is associated with a set of time indices `inds`, which specifies the time steps for which the knotpoint applies.
* `ConstraintList` only stores the constraint definition. It does not store constraint values and therefore cannot be evaluated. This is the representation stored in `Problem`.
* `AbstractConstraintSet` is meant to instantiated by the solver, and will usually be a simple wrapper around a vector of `AbstractConstraintValues`.
* The `AbstractConstraintValues` type is a wrapper around an `AbstractConstraint` and adds storage for constraint values, Jacobians, and anything else needed by the solver when dealing with constraints. This can implemented by the solver, or the solver can use the built-in `ConVal` type, demonstrated below.

## Stage Constraints
Let's start by looking at the `CircleConstraint` we created:
```julia
CircleConstraint(n, SA_F64[1,2], SA_F64[1,2], SA[0.1,0.1])
```
Note that this is a `StateConstraint`, meaning it is only a function of the state at a single time step. We also show a few methods for extracting information from the constraint.

In [19]:
con,inds = cons[2], 1:N-1
@show typeof(con)                     # just verifying it's a CircleConstraint
@test con isa TO.StateConstraint      # it inherits from StateConstraint, so it's a function of a single state
@test state_dim(con) == n             # the state dimension. control_dim won't be defined.
@test TO.check_dims(con, n, m)        # useful method to check if a constraint is consistent with the problem sizes
p = length(con);                      # get the length of the constraint vector
println("Length of the constraint: $p")

typeof(con) = CircleConstraint{2,Float64}
Length of the constraint: 2


This means that the `evaluate` can be passed a single state vector. Alternatively, we can pass a `KnotPoint` and the state will be extracted automatically:

In [20]:
z = prob.Z[end-1]
x = state(z)
v1 = TO.evaluate(con, x)
v2 = TO.evaluate(con, z) 
@test v1 == v2
v1

2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
 -0.99
 -0.99

For the constraint Jacobian, since it's only a function of the state, we'd expect it to be of size `(p,n)`. A `StageConstraint` will have a size of `(p,n+m)`, and `ControlConstraint` will have a size of `(p,m)`. We can generate the Jacobian automatically using `TO.gen_jacobian`:

In [21]:
jac = TO.gen_jacobian(con)

2×13 SizedArray{Tuple{2,13},Float64,2,2,Array{Float64,2}} with indices SOneTo(2)×SOneTo(13):
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

We can now evaluate the constraint using `TO.jacobian!`. We can pass in either the state or a `KnotPoint`. As with cost Hessians, the constraint Jacobian returns a boolean specifying whether the Jacobian is constant or not (i.e. the constraint is linear).

In [22]:
jacobian!(jac, con, x)
jacobian!(jac, con, z)

false

## Dynamics Constraints
TrajectoryOptimization also naturally has to deal with dynamics constraints, which couple states and controls across 2 adjacent time steps. Let's start by creating a dynamics constraint using the built-in `TO.add_dynamics_constraint!`. Internally, this is just creating a `DynamicsConstraint` and a `GoalConstraint` for the initial condition, calling the following constructor the `DynamicsConstraint`.
```julia
dyn_con = DynamicsConstraint{TO.integration(prob)}(prob.model, prob.N)
```

In [23]:
prob2 = copy(prob)
TO.add_dynamics_constraints!(prob2)
println("Number of constraint before: ", length(TO.get_constraints(prob)))
println("Number of constraint after: ", length(TO.get_constraints(prob2)))

dyn = prob2.constraints[end]
println("Type of dynamics constraint:")
println(typeof(dyn))  # type holds integration method, model type, and model sizes

Number of constraint before: 3
Number of constraint after: 5
Type of dynamics constraint:
TrajectoryOptimization.DynamicsConstraint{RK3,Quadrotor{UnitQuaternion{Float64}},13,4,17,Float64}


To evaluate the constraint, we need to pass in the states at the current and next time steps, along with the control at the current time step. We simplify this by simply requiring that we pass in the `KnotPoint`s for the current and next time step. As expected, the dynamics error for our rolled-out trajectory should be zero:

In [24]:
z2 = prob.Z[end]
TO.evaluate(dyn, z, z2)'

1×13 Adjoint{Float64,SArray{Tuple{13},Float64,1,13}} with indices SOneTo(1)×SOneTo(13):
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

The Jacobian gets a little trickier, since we need to separate the Jacobian between time steps. Our Jacobian looks something like:
$$ [A \; B \;\vert\; -I] $$
TrajectoryOptimization splits this by time step:
$$ [\nabla f_1 \;\vert\; \nabla f_2] $$

To initialize the Jacobians, we can use the generic `TO.widths` which returns a tuple of "widths," meaning the number of columns in each partition. Since our first partition has both state and controls, and the second only has states, we get:

In [25]:
w = TO.widths(dyn)

(17, 13)

We can use this to initialize our Jacobians for both steps. When we call `jacobian!` we need to specify which of the time steps we're taking the Jacobian with respect to.

In [26]:
jac1 = zeros(n,w[1])  # can also use TO.gen_jacobian(dyn)
jac2 = zeros(n,w[2])
println("Is constant? ", jacobian!(jac1, dyn, z, z2, 1))
println("Is constant? ", jacobian!(jac2, dyn, z, z2, 2))
display(jac1)
display(jac2);

LoadError: MethodError: no method matching _discrete_jacobian!(::RobotDynamics.ForwardAD, ::Type{RK3}, ::Array{Float64,2}, ::Quadrotor{UnitQuaternion{Float64}}, ::RobotDynamics.GeneralKnotPoint{Float64,13,4,SArray{Tuple{17},Float64,1,17}}, ::FiniteDiff.JacobianCache{Array{Float64,1},Array{Float64,1},Array{Float64,1},UnitRange{Int64},Nothing,Val{:forward}(),Float64})
Closest candidates are:
  _discrete_jacobian!(::RobotDynamics.ForwardAD, ::Type{Q}, ::Any, ::AbstractModel, ::AbstractKnotPoint{T,N,M}) where {T, N, M, Q<:RobotDynamics.Explicit} at /home/brian/.julia/packages/RobotDynamics/yaan9/src/model.jl:246
  _discrete_jacobian!(!Matched::RobotDynamics.FiniteDifference, ::Type{Q}, ::Any, ::AbstractModel, ::AbstractKnotPoint{T,N,M}, ::Any) where {T, N, M, Q<:RobotDynamics.Explicit} at /home/brian/.julia/packages/RobotDynamics/yaan9/src/model.jl:255
  _discrete_jacobian!(!Matched::RobotDynamics.FiniteDifference, ::Type{Q}, ::Any, ::AbstractModel, ::AbstractKnotPoint{T,N,M}) where {T, N,

## `AbstractConstraintValues` type
The `AbstractConstraintValues` type is meant to store all of the constraint values and Jacobians. We'll use the provided `ConVal` instantiation here. This type allows us to change the underlying storage type of our constraint values and Jacobians, allowing us to either use small dense arrays, or views into a large, sparse matrix. We show how to do both below:

In [27]:
cons = prob2.constraints                      # get `ConstraintList` with dynamics constraint
circle,inds = cons[3], cons.inds[3]
dyn = cons[end]

### Dense Matrix Storage ###

# to allocate our storage, we can use the provided `TO.gen_convals` method
p = length(circle)
C,c = TO.gen_convals(n,m,circle,inds)
@test length(C) == length(c) == length(inds)  # all these should be same length, the number of time steps
@test size(C[1]) == (p,n)                     # each element in a Jacobian
@test typeof(C[1]) <: SizedMatrix{p,n}        # each element in a SizedMatrix

# we can now generate our ConVal type
cval = TO.ConVal(n, m, circle, inds, C, c)


### Sparse Matrix Storage ###
using SparseArrays

# allocate sparse array and vector
NN = TO.num_vars(prob.Z)                      # total number of states and control (decision variables)
P = sum(TO.num_constraints(cons))             # total number of constraints
D = spzeros(P,NN)
d = zeros(P)

# to specify the ordering of the constraints, we use the provided `JacobianStructure` type
jac_struct = TO.JacobianStructure(cons, :by_knotpoint)  # 2nd argument optional. Other option is :by_constraint

# we can impart the structure to our sparse matrix using
D_struct = spzeros(Int,P,NN)
TO.jacobian_structure!(D_struct, jac_struct)

# with our structure specified, we can now create views for all of our constraints at the same time
C,c = TO.gen_convals(D,d,cons,jac_struct)

# this output is a little different than the previous one, since it's for all of the constraints
#   let's look at the output for the CircleConstraint (index 3)
@test length(C) == length(cons)               # length of output is the number of constraints
@test length(C[3]) == length(inds)            # length of each element is the number of time steps
@test size(C[3][1]) == (p,n)                  # each sub-element is a Jacobian

# for the DynamicsConstraint it's slightly different
@test size(C[end]) == (50,2)                  # the Jacobians are stored in a Matrix
@test size(c[end]) == (50,)                   # the constraint values are still stored in a Vector
@test size(C[end][1,1]) == (n,n+m)            # the 1st column is the Jacobian for the current time step
@test size(C[end][1,2]) == (n,n)              # the 2nd column is the Jacobian for the next time step
@test typeof(C[end][1]) <: SubArray           # each Jacobian is a view into D
@test typeof(c[end][1]) <: SubArray           # each constraint value is a view into d

# let's verify the views worked
c[end][1] .= 1
@test d[jac_struct.cinds[end][1]] == ones(n)  # the jac_struct stores the indices we need
C[end][2,1] .= 2
@test D[jac_struct.cinds[end][2], jac_struct.zinds[end][2]] == fill(2,n,n+m)
C[end][2,2] .= 3
@test D[jac_struct.cinds[end][2], jac_struct.zinds[end][3]] == [fill(3,n,n) zeros(n,m)]

# we can now generate a list of ConVals
cvals = map(1:length(cons)) do i
    TO.ConVal(n,m,cons[i],cons.inds[i],C[i],c[i])
end;

With an `AbstractContraintValues` type, we're now provided a few convenient methods for evaluating our constraint:

In [29]:
# Calculate the constraint values
TO.evaluate!(cval, prob.Z)

# Calculate the constraint Jacobian
TO.jacobian!(cval, prob.Z)

# Calcualate the maximum constraint violation
max_violation(cval)

# Individual constraint values and Jacobians are accessed via the `vals` and `jac` fields
cval.vals[2]   # constraint value at 2nd time step
cval.jac[2,1]  # constraint Jacobian at the 2nd time step, for the current time step

2×13 SizedArray{Tuple{2,13},Float64,2,2,Array{Float64,2}} with indices SOneTo(2)×SOneTo(13):
 -0.0  -2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  2.0  -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

This is easier than calling the underlying methods that work across all time steps. Note that these methods can be helpful to know about, since they can be overloaded for a custom constraint that needs to change behavior at different time steps.

In [30]:
TO.evaluate!(cval.vals, circle, prob.Z, inds)
TO.jacobian!(cval.jac, circle, prob.Z, inds, cval.is_const)

## `AbstractConstraintSet`
Now that we're familiar with the `AbstractConstraintValues` type, we can basically put these together into a vector and call it an `AbstractConstraintSet`. We'll create a minimal constraint set here.


In [33]:
struct BasicConstraintSet{T} <: TO.AbstractConstraintSet
    convals::Vector{TO.ConVal}
    c_max::Vector{T}
end
@inline TO.get_convals(conSet::BasicConstraintSet) = conSet.convals

conSet = BasicConstraintSet(Vector{TO.ConVal}(cvals), zeros(length(cvals)));

In [39]:
@test length(conSet) == length(cons)  # supports iteration (currently indexing is not supported)

# supports the same methods we used on `ConVal`, but just applies them to all constraints
TO.evaluate!(conSet, prob.Z)
TO.jacobian!(conSet, prob.Z)
max_violation(conSet)

2.0